In [1]:
import json

In [2]:
json_file = "/home/dola/data/works.json"
jsonl_file = "raw_data.jsonl"

In [3]:
with open(json_file, 'r') as file:
    data = json.load(file)
len(data)

842966

In [5]:
out_data = []

system_text = "你是一名文言文翻译家，能够将文言文准确优雅地翻译成现代白话文。"
input_ct_prefix = "请将下述文言文翻译成现代白话文："

for item in data:
    k = item["Kind"]
    c = item['Content']
    t = item['Translation']

    if c == None or t == None:continue
    if len(c) <= 1 or len(t) <= 1:continue
    if c.find("【") != -1:continue
    
    c_list = c.split("\r\n")
    t_list = t.split("\r\n")
    if len(c_list) != len(t_list):
        ct_conversation = {"system": system_text,
                        "input": input_ct_prefix + c.replace("\r\n","").replace("\n",""),
                        "output": t.replace("\r\n","").replace("\n","")}
        out_data.append({"conversation": [ct_conversation]})

    else:
        for i,j in zip(c_list,t_list):
            ct_conversation = {"system": system_text,
                            "input": input_ct_prefix + i.replace("\r\n","").replace("\n",""),
                            "output": j.replace("\r\n","").replace("\n","")}            
            out_data.append({"conversation": [ct_conversation]})


In [6]:
print(len(out_data))

20303


In [7]:
import json
with open(jsonl_file, 'w', encoding='utf-8') as f:
    for item in out_data:
        json_str = json.dumps(item, ensure_ascii=False)
        f.write(json_str+"\n")

## 划分

In [8]:
# copy from https://github.com/InternLM/tutorial/blob/main/xtuner/split2train_and_test.py and modified

import random
import jsonlines
def split_conversations(input_file, train_output_file, test_output_file):
    data = []
    with open(input_file) as file:
        for item in jsonlines.Reader(file):
            data.append(item)

    num_conversations = len(data)
    
    random.shuffle(data)
    random.shuffle(data)
    random.shuffle(data)
    split_point = int(num_conversations * 0.95)

    train_data = data[:split_point]
    test_data = data[split_point:]

    print(f"train data szie = {len(train_data)},train data example = {train_data[0]}")
    print(f"test data szie = {len(test_data)},test data example = {test_data[0]}")

    with open(train_output_file, 'w', encoding='utf-8') as f:
        for item in train_data:
            json_str = json.dumps(item, ensure_ascii=False)
            f.write(json_str+"\n")

    with open(test_output_file, 'w', encoding='utf-8') as f:
        for item in test_data:
            json_str = json.dumps(item, ensure_ascii=False)
            f.write(json_str+"\n")

    print(f"Split complete. Train data written to {train_output_file}, Test data written to {test_output_file}")

split_conversations(jsonl_file, "train.jsonl", "test.jsonl")

train data szie = 19287,train data example = {'conversation': [{'system': '你是一名文言文翻译家，能够将文言文准确优雅地翻译成现代白话文。', 'input': '请将下述文言文翻译成现代白话文：翩翩之燕，远集西羌。', 'output': '翩翩起舞的紫燕，飞向那遥远的西羌。'}]}
test data szie = 1016,test data example = {'conversation': [{'system': '你是一名文言文翻译家，能够将文言文准确优雅地翻译成现代白话文。', 'input': '请将下述文言文翻译成现代白话文：霜落熊升树，林空鹿饮溪。', 'output': '太阳高升，霜雪融落，山林显得愈加寂静空荡，笨熊正在缓慢地爬着大树，鹿儿正在悠闲地喝着小溪的潺潺流水。'}]}
Split complete. Train data written to train.jsonl, Test data written to test.jsonl
